_TODO_ Don't forget to add Graphulo jars to your server-side Accumulo files! More info here: _TODO_ 

In [1]:
print("is restarted? this ought to be cell 1")

is restarted? this ought to be cell 1

In [2]:
using Revise
using D4M

┌ Info: Precompiling D4M [ca196bdc-a701-11e8-3d50-3b5cc8577617]
└ @ Base loading.jl:1192


Database capabilities loaded!


In [3]:
testD4Mver()

v test 6


In [4]:
dbinit()

In [6]:
DB = dbsetup("uno", "AccumuloConfig.jl")

26 Jun 2019 09:29:56,648 WARN - ClientConfiguration.loadFromSearchPath(227) -  Found no client.conf in default paths. Using default client configuration values.


D4M.DBserver("uno", "localhost:2181", "root", "secret", "BigTableLike", JavaCall.JavaObject{Symbol("edu.mit.ll.graphulo.MatlabGraphulo")}(Ptr{Nothing} @0x00000000051639e8))

In [7]:
A = DB["mytable_A"]
B = DB["mytable_B"]

ls(DB)

Creating mytable_A in uno
Creating mytable_B in uno


6-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "mytable_A"           
 "mytable_B"           
 "trace"               

In [8]:
row = [1,1,2,2]
column = [1,2,1,2]
values = [1,2,3,4]

AssocA = Assoc(row,column,values)
put(A, AssocA)
printFull(A[:,:])

row = [1,1,2,2]
column = [1,2,1,2]
values = [5,6,7,8]

AssocB = Assoc(row,column,values)
put(B, AssocB)
printFull(B[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "1"  "2"
 "2"  "3"  "4"

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "5"  "6"
 "2"  "7"  "8"

In [17]:
C = tablemult(A, B, "mytable_C")

26 Jun 2019 09:33:34,462 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 8 entries processed


D4M.DBtable(D4M.DBserver("uno", "localhost:2181", "root", "secret", "BigTableLike", JavaCall.JavaObject{Symbol("edu.mit.ll.graphulo.MatlabGraphulo")}(Ptr{Nothing} @0x00000000051639e8)), "mytable_C", "", 0, 0, "", 500000.0, JavaCall.JavaObject{Symbol("edu.mit.ll.d4m.db.cloud.D4mDataSearch")}(Ptr{Nothing} @0x00000000051639d0), JavaCall.JavaObject{Symbol("edu.mit.ll.d4m.db.cloud.D4mDbTableOperations")}(Ptr{Nothing} @0x0000000005163a58))

In [19]:
printFull(C[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "26"  "30"
 "2"  "38"  "44"